In [142]:
import pandas as pd
import seaborn as sns
import altair as alt

# WAYGCA MODEL
### Read in data

In [50]:
raw_data = pd.read_csv("/Users/bs/Projects/R_F_GitHub/nyc-complaints/data/clean_merge.csv")

### Set up Y values by filtering most common complaint types

Grouping complaint types as follows:

"Noise - Outside" = ["Noise - Street/Sidewalk", "Noise - Commercial", "Noise", "Noise - Vehicle", "Noise - Helicopter" ]

"Apartment Construction/Maintenance" = ["PAINT/PLASTER","DOOR/WINDOW", "General Construction/Plumbing", "APPLIANCE"]

"Water Leaks and Plumbing" =["PLUMBING", "Water System", "WATER LEAK", "Water Quality","Sewer", "Plumbing"]

"Building Maintenance" = ["FLOORING/STAIRS","Elevator","Building/Use","Maintenance or Facility"]

"Heat/Hot Water/Electricity" = ["HEAT/HOT WATER", "ELECTRIC"]

"Noise - Residential" = ["Noise - Residential"]

"Parking/Street Conditions" = ["Illegal Parking","Street Condition", "Blocked Driveway","Graffiti","Sidewalk Condition","Street Light Condition","Dead/Dying Tree"

"Unhealthy Conditions" = ["Unsanitary Condition", "Lead", "Dirty Conditions", "Air Quality", "Indoor Air Quality","SAFETY","Sanitation Condition"]

Final Complaint List = ["Rodent","Unhealthy Conditions","Parking/Street Conditions","Noise - Residential","Heat/Hot Water/Electricity","Building Maintenance","Water Leaks and Plumbing","Apartment Construction/Maintenance","Noise - Outside"]

In [51]:
data = raw_data

In [52]:
out_noise = ["Noise - Street/Sidewalk", "Noise - Commercial", "Noise", "Noise - Vehicle", "Noise - Helicopter" ]
apt = ["PAINT/PLASTER","DOOR/WINDOW", "General Construction/Plumbing", "APPLIANCE"]
water =["PLUMBING", "Water System", "WATER LEAK", "Water Quality","Sewer", "Plumbing"]
bldg = ["FLOORING/STAIRS","Elevator","Building/Use","Maintenance or Facility"]
heat = ["HEAT/HOT WATER", "ELECTRIC"]
noise = ["Noise - Residential"]
parking = ["Illegal Parking","Street Condition", "Blocked Driveway","Graffiti","Sidewalk Condition","Street Light Condition","Dead/Dying Tree"]
health = ["Unsanitary Condition", "Lead", "Dirty Conditions", "Air Quality", "Indoor Air Quality","SAFETY","Sanitation Condition"]

In [53]:
data = data.replace(to_replace = out_noise, value = "Noise - Outside")
data = data.replace(to_replace = apt, value = "Apartment Construction/Maintenance")
data = data.replace(to_replace = water, value = "Water Leaks and Plumbing")
data = data.replace(to_replace = bldg, value = "Building Maintenance")
data = data.replace(to_replace = heat, value = "Heat/Hot Water/Electricity")
data = data.replace(to_replace = noise, value = "Noise - Residential")
data = data.replace(to_replace = parking, value = "Parking/Street Conditions")
data = data.replace(to_replace = health, value = "Unhealthy Conditions")

In [54]:
data.shape

(62350, 10)

In [55]:
complaints_to_keep = ["st_address_lower","Rodent","Unhealthy Conditions","Parking/Street Conditions","Noise - Residential","Heat/Hot Water/Electricity","Building Maintenance","Water Leaks and Plumbing","Apartment Construction/Maintenance","Noise - Outside"]
filtered_data = data[data["Complaint Type"].isin(complaints_to_keep)]


In [56]:
filtered_data.head()

,Unnamed: 0,st_address_lower,Complaint Type,zipcode,Units,Stories,Built,unit_type,neighborhood,Popularity
3,3,1774 amsterdam avenue,Noise - Outside,10031,28.0,5,1930,Rental Building,Hamilton Heights,23
5,5,68 bayard street,Unhealthy Conditions,10013,16.0,6,1915,Rental Building,Chinatown,6
6,6,120 haven avenue,Noise - Residential,10032,36.0,5,1920,Rental Building,Washington Heights,18
7,7,120 haven avenue,Noise - Residential,10032,36.0,5,1920,Rental Building,Washington Heights,18
8,8,120 haven avenue,Noise - Residential,10032,36.0,5,1920,Rental Building,Washington Heights,18


### Address imbalanced class sizes (todo)

In [131]:
filtered_data_counts = filtered_data.groupby(["Complaint Type"]).agg(count_col= pd.NamedAgg(column="Complaint Type", aggfunc="count")).reset_index()

In [138]:
filtered_data_counts["percent"] = filtered_data_counts["count_col"].apply(lambda x: x*100/(filtered_data_counts["count_col"].sum()))

In [141]:
filtered_data_counts.sort_values(by = 'percent', ascending = False)

,Complaint Type,count_col,percent
3,Noise - Outside,13550,28.626962
2,Heat/Hot Water/Electricity,9503,20.076902
4,Noise - Residential,7724,16.318425
5,Parking/Street Conditions,5900,12.464877
0,Apartment Construction/Maintenance,3457,7.303573
8,Water Leaks and Plumbing,3270,6.908499
7,Unhealthy Conditions,1910,4.035240
1,Building Maintenance,1279,2.702132
6,Rodent,740,1.563391


### Set up X values by converting to float

In [57]:
filtered_data["zipcode"] = filtered_data["zipcode"].apply(str)
filtered_data["Units"] = filtered_data["Units"].apply(float)
filtered_data["Popularity"] = filtered_data["Popularity"].apply(lambda x: float(x.replace(",","")) if type(x)==str else float(x))
filtered_data["Stories"] = filtered_data["Stories"].apply(lambda x: float(x) if type(x)==float or x.isnumeric() else None)
filtered_data["Built"] = filtered_data["Built"].apply(lambda x: float(x) if type(x)==float or x.isnumeric() else None)

<ipython-input-57-e12330141406>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["zipcode"] = filtered_data["zipcode"].apply(str)
<ipython-input-57-e12330141406>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_data["Units"] = filtered_data["Units"].apply(float)
<ipython-input-57-e12330141406>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

### Save address lookup table

In [58]:
cols_to_keep_lookup = ["st_address_lower","Complaint Type", "Units", "Stories","Built","Popularity", "zipcode","neighborhood","unit_type"]
lookup_data = filtered_data[cols_to_keep_lookup]
lookup_data.to_csv("/Users/bs/Projects/R_F_GitHub/nyc-complaints/data/lookup_data.csv")

In [10]:
cols_to_keep = ["Complaint Type", "Units", "Stories","Built","Popularity", "zipcode","neighborhood","unit_type"]
final_data = filtered_data[cols_to_keep]

In [11]:
y = final_data["Complaint Type"]

In [12]:
X = final_data.drop(labels = "Complaint Type", axis = 1)

In [13]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

transformer_name = 'ohe_on_all_categorical_features'
transformer = OneHotEncoder(sparse=False)
columns_to_encode = ['zipcode', 'neighborhood', 'unit_type']

ohe_final = ColumnTransformer([
    (transformer_name, transformer, columns_to_encode)], 
    remainder='passthrough')

imputer = SimpleImputer(strategy = 'median', fill_value = 0)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of the complete data set:", X.shape)
print("Shape of the train data set:", X_train.shape)
print("Shape of the test data set:", X_test.shape)

Shape of the complete data set: (47333, 7)
Shape of the train data set: (37866, 7)
Shape of the test data set: (9467, 7)


In [16]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(LogisticRegression(max_iter = 1000), cv = 5, param_grid = {'C':[1,2]})
lr_pipe = Pipeline([('ohe', ohe_final),
                    ('imputer', imputer),
                    ('scaler', StandardScaler()),
                    ('gs', grid_search)])
                           
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('ohe',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_on_all_categorical_features',
                                                  OneHotEncoder(sparse=False),
                                                  ['zipcode', 'neighborhood',
                                                   'unit_type'])])),
                ('imputer', SimpleImputer(fill_value=0, strategy='median')),
                ('scaler', StandardScaler()),
                ('gs',
                 GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
                              param_grid={'C': [1, 2]}))])

In [17]:
from sklearn import metrics

def model_evaluation(model, X, y_true):
    y_pred = model.predict(X)
    scores = {}
    scores['accuracy'] = round(metrics.accuracy_score(y_true, y_pred), 4)
    scores['precision'] = round(metrics.precision_score(y_true, y_pred, average='weighted'), 4)
    scores['recall'] = round(metrics.recall_score(y_true, y_pred, average='weighted'), 4)
    probs = model.predict_proba(X).T[1]
    #scores['log_loss'] = round(metrics.log_loss(y_true,probs),4)
    return scores

def print_model_evaluation(model_name, scores):
    print('{} evaluation \n'.format(model_name))
    for metric, score in scores.items():
        print('Test {}: {}'.format(metric, score))

In [18]:
lr_pipe_score = model_evaluation(lr_pipe, X_test, y_test)
print_model_evaluation('Logistic regression', lr_pipe_score)

Logistic regression evaluation 

Test accuracy: 0.3914
Test precision: 0.3273
Test recall: 0.3914


/Users/bs/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
import dill

with open('pipe_of_logreg.dill', 'wb') as f:
    dill.dump(lr_pipe, f)

In [60]:
with open('pipe_of_logreg.dill', 'rb') as f:
    lr_pipe = dill.load(f)

In [62]:
lr_pipe.predict(X_test)

array(['Noise - Outside', 'Heat/Hot Water/Electricity', 'Noise - Outside',
       ..., 'Heat/Hot Water/Electricity', 'Noise - Outside',
       'Noise - Residential'], dtype=object)

In [34]:
probs = lr_pipe.predict_proba(X_test)

False

In [95]:
test={"Units":[25.0], "Stories":[6.0],"Built":[1907.0],"Popularity":[3.0], "zipcode":["10032"], "neighborhood":["Washington Heights"],"unit_type":["Rental Building"]}

In [96]:
test_df = pd.DataFrame.from_dict(test)

In [97]:
test_df

,Units,Stories,Built,Popularity,zipcode,neighborhood,unit_type
0,25.0,6.0,1907.0,3.0,10032,Washington Heights,Rental Building


In [118]:
X_axis_probs = [round(x,2) for x in lr_pipe.predict_proba(test_df).flatten().tolist()]

In [119]:
X_axis_probs

[0.09, 0.04, 0.3, 0.3, 0.13, 0.04, 0.01, 0.02, 0.08]

In [120]:
Y_axis = lr_pipe.classes_.flatten().tolist()

In [121]:
Y_axis

['Apartment Construction/Maintenance',
 'Building Maintenance',
 'Heat/Hot Water/Electricity',
 'Noise - Outside',
 'Noise - Residential',
 'Parking/Street Conditions',
 'Rodent',
 'Unhealthy Conditions',
 'Water Leaks and Plumbing']

In [126]:
predictions = pd.DataFrame(zip(Y_axis,X_axis_probs), columns = ["Complaint Type", "Probability"])

In [129]:
predictions.sort_values( by = "Probability", ascending = False)

,Complaint Type,Probability
2,Heat/Hot Water/Electricity,0.30
3,Noise - Outside,0.30
4,Noise - Residential,0.13
0,Apartment Construction/Maintenance,0.09
8,Water Leaks and Plumbing,0.08
1,Building Maintenance,0.04
5,Parking/Street Conditions,0.04
7,Unhealthy Conditions,0.02
6,Rodent,0.01


### First Logistic Regression Model without grid search
Logistic regression evaluation 

Test accuracy: 0.3914
Test precision: 0.3273
Test recall: 0.3914

### Second Logistic Regression Model with grid search
Logistic regression evaluation 

Test accuracy: 0.3889
Test precision: 0.3326
Test recall: 0.3889

### Logistic Regression with just zipcode

In [160]:
y = final_data["Complaint Type"]
X_zip = final_data.drop(labels = ["Complaint Type","Units","Stories","Popularity","Built","unit_type","neighborhood"], axis = 1)

In [161]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

transformer_name = 'ohe_on_all_categorical_features'
transformer = OneHotEncoder(sparse=False)
columns_to_encode = ['zipcode']

ohe_final = ColumnTransformer([
    (transformer_name, transformer, columns_to_encode)], 
    remainder='passthrough')

imputer = SimpleImputer(strategy = 'median', fill_value = 0)

In [163]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_zip, y, test_size=0.2, random_state=42)

print("Shape of the complete data set:", X.shape)
print("Shape of the train data set:", X_train.shape)
print("Shape of the test data set:", X_test.shape)

Shape of the complete data set: (47333, 7)
Shape of the train data set: (37866, 1)
Shape of the test data set: (9467, 1)


In [164]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(LogisticRegression(max_iter = 1000), cv = 5, param_grid = {'C':np.logspace(-2,2,20)})
lr_pipe = Pipeline([('ohe', ohe_final),
                    ('imputer', imputer),
                    ('scaler', StandardScaler()),
                    ('gs', grid_search)])
                           
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('ohe',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_on_all_categorical_features',
                                                  OneHotEncoder(sparse=False),
                                                  ['zipcode'])])),
                ('imputer', SimpleImputer(fill_value=0, strategy='median')),
                ('scaler', StandardScaler()),
                ('gs',
                 GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=1000),
                              param_grid={'C': array([1.00000000e-02, 1.62377674e-02, 2.63665090e-02, 4.28133240e-02,
       6.95192796e-02, 1.12883789e-01, 1.83298071e-01, 2.97635144e-01,
       4.83293024e-01, 7.84759970e-01, 1.27427499e+00, 2.06913808e+00,
       3.35981829e+00, 5.45559478e+00, 8.85866790e+00, 1.43844989e+01,
       2.33572147e+01, 3.79269019e+01, 6.15848211e+01, 1.00000000e+02])}))])

In [165]:
from sklearn import metrics

def model_evaluation(model, X, y_true):
    y_pred = model.predict(X)
    scores = {}
    scores['accuracy'] = round(metrics.accuracy_score(y_true, y_pred), 4)
    scores['precision'] = round(metrics.precision_score(y_true, y_pred, average='weighted'), 4)
    scores['recall'] = round(metrics.recall_score(y_true, y_pred, average='weighted'), 4)
    probs = model.predict_proba(X).T[1]
    return scores

def print_model_evaluation(model_name, scores):
    print('{} evaluation \n'.format(model_name))
    for metric, score in scores.items():
        print('Test {}: {}'.format(metric, score))

In [166]:
lr_pipe_score = model_evaluation(lr_pipe, X_test, y_test)
print_model_evaluation('Logistic regression', lr_pipe_score)

Logistic regression evaluation 

Test accuracy: 0.3192
Test precision: 0.256
Test recall: 0.3192


/Users/bs/opt/anaconda3/envs/capstone/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Zipcode Model Results
Logistic regression evaluation 

Test accuracy: 0.3192
Test precision: 0.256
Test recall: 0.3192